In [1]:
#Based on this tutorial - http://sebastianraschka.com/Articles/2015_pca_in_3_steps.html

import pandas as pd
import pylab as pl
import numpy as np 
import geopandas as gp
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Data munging for all DV routes

In [2]:
demographics = pd.read_excel('../data/DemoData.xlsx')
demographics['tract'] = demographics.ID.astype(str).str.zfill(9)

In [3]:
shp = gp.GeoDataFrame.from_file('../data/merged.json')

shp = shp.merge(demographics, on='tract')

shp.rename(columns={
    'Destination?': 'is_dest',
    'Origin?': 'is_orig',
    'Total Population': 'pop',
    '18 to 24 yrs': '_18_to_24',
    '65 and Above': 'above_65',
    'Median Age': 'age',
    'Non-white': 'nonwhite',
    'English less than "very well"': 'nonenglish',
    'Annaul Individual Income below 10000': 'income_below_10000',
    'Annaul Individual Income below 15000': 'income_below_15000',
    'Median Income': 'income',
    'Below 100 percent of the poverty level': 'below_pov',
    '100 to 149 percent of the poverty level': '_100_149_pov',
    'At or above 150 percent of the poverty level': 'above_150_pov',
    'Citizenship status': 'noncitizen',
    'Place of Birth': 'foreignborn'
}, inplace=True)

In [4]:
lehd = pd.read_csv('../data/merged.csv', dtype={'tract': str})

lehd = lehd.drop([u'Unnamed: 0', u'Both', u'Destination?', u'DollarVanLine', u'Origin?',
       u'average_commute_time', u'average_walk_distance', u'geometry',
       u'number_of_commuters',  u'w_county_tract', u'h_county_tract'], 1)

shp = shp.merge(lehd, on='tract')

In [5]:
shp.dropna(how="all", inplace=True)

In [6]:
routes = pd.read_csv('../data/csv/vanroutes3-11_430pm.csv', dtype={'CT2010': str, 'BoroCode': str})

routes['county_code'] = ""
routes['county_code'][routes['BoroCode'] == '1'] = '061' # Manhattan
routes['county_code'][routes['BoroCode'] == '2'] = '005' # Bronx
routes['county_code'][routes['BoroCode'] == '3'] = '047' # Brooklyn
routes['county_code'][routes['BoroCode'] == '4'] = '081' # Queens
routes['county_code'][routes['BoroCode'] == '5'] = '085' # Staten Island

routes['tract'] = routes['county_code'] + routes['CT2010']

routes.drop(['FID',
 'FID_1',
 'CTLabel',
 'BoroCode',
 'BoroName',
 'CT2010',
 'BoroCT2010',
 'CDEligibil',
 'NTACode',
 'NTAName',
 'PUMA',
 'Shape_Leng',
 'Shape_Area',
 'Count_'
], 1, inplace = True)



/Library/Python/2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Python/2.7/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Python/2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Library/Python/2.7/site-packages/ipykernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the the caveat

In [7]:
shp = shp.merge(routes, on = 'tract')

In [8]:
shp.columns

Index([                 u'Both',               u'is_dest',
               u'DollarVanLine',               u'is_orig',
        u'average_commute_time', u'average_walk_distance',
                    u'geometry',   u'number_of_commuters',
                       u'tract',                    u'ID',
                   u'Geography',                   u'pop',
                   u'_18_to_24',              u'above_65',
                         u'age',              u'nonwhite',
                  u'nonenglish',    u'income_below_10000',
          u'income_below_15000',                u'income',
                   u'below_pov',          u'_100_149_pov',
               u'above_150_pov',            u'noncitizen',
                 u'foreignborn',        u'wac_total_jobs',
              u'wac_jobs_lt_29',        u'wac_jobs_30_54',
              u'wac_jobs_gt_55',      u'wac_minc_lt_1250',
          u'wac_minc_1251_3333',      u'wac_minc_gt_3333',
              u'rac_total_jobs',        u'rac_jobs_lt_29

### Individual routes
#### Chinatown

In [9]:
#Origins

china_data_o = shp.ix[:,4:]
labels = shp['DollarVanLine']
china_data_o = shp[routes['OriginChina'] == 1]

china_data_o = china_data_o[[
        u'average_commute_time', u'average_walk_distance',
        u'number_of_commuters',u'pop',
                       u'_18_to_24',                    u'above_65',
                         u'age',             u'nonwhite',
                  u'nonenglish',    u'income_below_10000',
          u'income_below_15000',                u'income',
                  u'noncitizen',           u'foreignborn',
              u'wac_total_jobs',
              u'wac_jobs_lt_29',        u'wac_jobs_30_54',
              u'wac_jobs_gt_55',      u'wac_minc_lt_1250',
          u'wac_minc_1251_3333',      u'wac_minc_gt_3333',
              u'rac_total_jobs',        u'rac_jobs_lt_29',
              u'rac_jobs_30_54',        u'rac_jobs_gt_55',
            u'rac_minc_lt_1250',    u'rac_minc_1251_3333',
            u'rac_minc_gt_3333'
]]

#Dropping poverty because nans while computing covariance matrix
china_data_o_std = china_data_o

# Standardize Value Range

for c in [
        u'average_commute_time', u'average_walk_distance',
        u'number_of_commuters',u'pop',
                       u'_18_to_24',                    u'above_65',
                         u'age',             u'nonwhite',
                  u'nonenglish',    u'income_below_10000',
          u'income_below_15000',                u'income',
                 u'noncitizen',
                 u'foreignborn',        u'wac_total_jobs',
              u'wac_jobs_lt_29',        u'wac_jobs_30_54',
              u'wac_jobs_gt_55',      u'wac_minc_lt_1250',
          u'wac_minc_1251_3333',      u'wac_minc_gt_3333',
              u'rac_total_jobs',        u'rac_jobs_lt_29',
              u'rac_jobs_30_54',        u'rac_jobs_gt_55',
            u'rac_minc_lt_1250',    u'rac_minc_1251_3333',
            u'rac_minc_gt_3333'
]:
    china_data_o_std[c] = ((china_data_o_std[c] - china_data_o_std[c].mean())/china_data_o_std[c].std())
    


/Library/Python/2.7/site-packages/pandas-0.16.2-py2.7-macosx-10.10-intel.egg/pandas/core/frame.py:1825: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  "DataFrame index.", UserWarning)


In [10]:
#Destination

china_data_d = shp.ix[:,4:]
labels = shp['DollarVanLine']
china_data_d = shp[routes['ChinaDest'] == 1]

china_data_d = china_data_d[[
        u'average_commute_time', u'average_walk_distance',
        u'number_of_commuters',u'pop',
                       u'_18_to_24',                    u'above_65',
                         u'age',             u'nonwhite',
                  u'nonenglish',    u'income_below_10000',
          u'income_below_15000',                u'income',
                  u'noncitizen',           u'foreignborn',
              u'wac_total_jobs',
              u'wac_jobs_lt_29',        u'wac_jobs_30_54',
              u'wac_jobs_gt_55',      u'wac_minc_lt_1250',
          u'wac_minc_1251_3333',      u'wac_minc_gt_3333',
              u'rac_total_jobs',        u'rac_jobs_lt_29',
              u'rac_jobs_30_54',        u'rac_jobs_gt_55',
            u'rac_minc_lt_1250',    u'rac_minc_1251_3333',
            u'rac_minc_gt_3333'
]]

#Dropping poverty because nans while computing covariance matrix
china_data_d_std = china_data_d

# Standardize Value Range

for c in [
        u'average_commute_time', u'average_walk_distance',
        u'number_of_commuters',u'pop',
                       u'_18_to_24',                    u'above_65',
                         u'age',             u'nonwhite',
                  u'nonenglish',    u'income_below_10000',
          u'income_below_15000',                u'income',
                 u'noncitizen',
                 u'foreignborn',        u'wac_total_jobs',
              u'wac_jobs_lt_29',        u'wac_jobs_30_54',
              u'wac_jobs_gt_55',      u'wac_minc_lt_1250',
          u'wac_minc_1251_3333',      u'wac_minc_gt_3333',
              u'rac_total_jobs',        u'rac_jobs_lt_29',
              u'rac_jobs_30_54',        u'rac_jobs_gt_55',
            u'rac_minc_lt_1250',    u'rac_minc_1251_3333',
            u'rac_minc_gt_3333'
]:
    china_data_d_std[c] = ((china_data_d_std[c] - china_data_d_std[c].mean())/china_data_d_std[c].std())
    


#### Flatbush

In [11]:
#Origins

flat_route_o = shp.ix[:,4:]
labels = shp['DollarVanLine']
flat_route_o = shp[routes['OriginFlat'] == 1]

flat_route_o = flat_route_o[[
        u'average_commute_time', u'average_walk_distance',
        u'number_of_commuters',u'pop',
                       u'_18_to_24',                    u'above_65',
                         u'age',             u'nonwhite',
                  u'nonenglish',    u'income_below_10000',
          u'income_below_15000',                u'income',
                  u'noncitizen',           u'foreignborn',
              u'wac_total_jobs',
              u'wac_jobs_lt_29',        u'wac_jobs_30_54',
              u'wac_jobs_gt_55',      u'wac_minc_lt_1250',
          u'wac_minc_1251_3333',      u'wac_minc_gt_3333',
              u'rac_total_jobs',        u'rac_jobs_lt_29',
              u'rac_jobs_30_54',        u'rac_jobs_gt_55',
            u'rac_minc_lt_1250',    u'rac_minc_1251_3333',
            u'rac_minc_gt_3333'
]]

#Dropping poverty because nans while computing covariance matrix
flat_route_o_std = flat_route_o

# Standardize Value Range

for c in [
        u'average_commute_time', u'average_walk_distance',
        u'number_of_commuters',u'pop',
                       u'_18_to_24',                    u'above_65',
                         u'age',             u'nonwhite',
                  u'nonenglish',    u'income_below_10000',
          u'income_below_15000',                u'income',
                 u'noncitizen',
                 u'foreignborn',        u'wac_total_jobs',
              u'wac_jobs_lt_29',        u'wac_jobs_30_54',
              u'wac_jobs_gt_55',      u'wac_minc_lt_1250',
          u'wac_minc_1251_3333',      u'wac_minc_gt_3333',
              u'rac_total_jobs',        u'rac_jobs_lt_29',
              u'rac_jobs_30_54',        u'rac_jobs_gt_55',
            u'rac_minc_lt_1250',    u'rac_minc_1251_3333',
            u'rac_minc_gt_3333'
]:
    flat_route_o_std[c] = ((flat_route_o_std[c] - flat_route_o_std[c].mean())/flat_route_o_std[c].std())
    



In [12]:
#Destination

flat_route_d = shp.ix[:,4:]
labels = shp['DollarVanLine']
flat_route_d = shp[routes['FlatbushDest'] == 1]

flat_route_d = flat_route_d[[
        u'average_commute_time', u'average_walk_distance',
        u'number_of_commuters',u'pop',
                       u'_18_to_24',                    u'above_65',
                         u'age',             u'nonwhite',
                  u'nonenglish',    u'income_below_10000',
          u'income_below_15000',                u'income',
                  u'noncitizen',           u'foreignborn',
              u'wac_total_jobs',
              u'wac_jobs_lt_29',        u'wac_jobs_30_54',
              u'wac_jobs_gt_55',      u'wac_minc_lt_1250',
          u'wac_minc_1251_3333',      u'wac_minc_gt_3333',
              u'rac_total_jobs',        u'rac_jobs_lt_29',
              u'rac_jobs_30_54',        u'rac_jobs_gt_55',
            u'rac_minc_lt_1250',    u'rac_minc_1251_3333',
            u'rac_minc_gt_3333'
]]

#Dropping poverty because nans while computing covariance matrix
flat_route_d_std = flat_route_d

# Standardize Value Range

for c in [
        u'average_commute_time', u'average_walk_distance',
        u'number_of_commuters',u'pop',
                       u'_18_to_24',                    u'above_65',
                         u'age',             u'nonwhite',
                  u'nonenglish',    u'income_below_10000',
          u'income_below_15000',                u'income',
                 u'noncitizen',
                 u'foreignborn',        u'wac_total_jobs',
              u'wac_jobs_lt_29',        u'wac_jobs_30_54',
              u'wac_jobs_gt_55',      u'wac_minc_lt_1250',
          u'wac_minc_1251_3333',      u'wac_minc_gt_3333',
              u'rac_total_jobs',        u'rac_jobs_lt_29',
              u'rac_jobs_30_54',        u'rac_jobs_gt_55',
            u'rac_minc_lt_1250',    u'rac_minc_1251_3333',
            u'rac_minc_gt_3333'
]:
    flat_route_d_std[c] = ((flat_route_d_std[c] - flat_route_d_std[c].mean())/flat_route_d_std[c].std())
    



#### Jamaica

In [28]:
#Origins

jam_route_o = shp.ix[:,4:]
labels = shp['DollarVanLine']
jam_route_o = shp[routes['OriginJamica'] == 1]

jam_route_o = jam_route_o[[
        u'average_commute_time', u'average_walk_distance',
        u'number_of_commuters',u'pop',
                       u'_18_to_24',                    u'above_65',
                         u'age',             u'nonwhite',
                  u'nonenglish',    u'income_below_10000',
          u'income_below_15000',                u'income',
                  u'noncitizen',           u'foreignborn',
              u'wac_total_jobs',
              u'wac_jobs_lt_29',        u'wac_jobs_30_54',
              u'wac_jobs_gt_55',      u'wac_minc_lt_1250',
          u'wac_minc_1251_3333',      u'wac_minc_gt_3333',
              u'rac_total_jobs',        u'rac_jobs_lt_29',
              u'rac_jobs_30_54',        u'rac_jobs_gt_55',
            u'rac_minc_lt_1250',    u'rac_minc_1251_3333',
            u'rac_minc_gt_3333'
]]

#Dropping poverty because nans while computing covariance matrix
jam_route_o_std = jam_route_o

# Standardize Value Range

for c in [
        u'average_commute_time', u'average_walk_distance',
        u'number_of_commuters',u'pop',
                       u'_18_to_24',                    u'above_65',
                         u'age',             u'nonwhite',
                  u'nonenglish',    u'income_below_10000',
          u'income_below_15000',                u'income',
                 u'noncitizen',
                 u'foreignborn',        u'wac_total_jobs',
              u'wac_jobs_lt_29',        u'wac_jobs_30_54',
              u'wac_jobs_gt_55',      u'wac_minc_lt_1250',
          u'wac_minc_1251_3333',      u'wac_minc_gt_3333',
              u'rac_total_jobs',        u'rac_jobs_lt_29',
              u'rac_jobs_30_54',        u'rac_jobs_gt_55',
            u'rac_minc_lt_1250',    u'rac_minc_1251_3333',
            u'rac_minc_gt_3333'
]:
    jam_route_o_std[c] = ((jam_route_o_std[c] - jam_route_o_std[c].mean())/jam_route_o_std[c].std())
    





In [29]:

jam_route_d = shp.ix[:,4:]
labels = shp['DollarVanLine']
jam_route_d = shp[routes['Jamica'] == 1]

jam_route_d = jam_route_d[[
        u'average_commute_time', u'average_walk_distance',
        u'number_of_commuters',u'pop',
                       u'_18_to_24',                    u'above_65',
                         u'age',             u'nonwhite',
                  u'nonenglish',    u'income_below_10000',
          u'income_below_15000',                u'income',
                  u'noncitizen',           u'foreignborn',
              u'wac_total_jobs',
              u'wac_jobs_lt_29',        u'wac_jobs_30_54',
              u'wac_jobs_gt_55',      u'wac_minc_lt_1250',
          u'wac_minc_1251_3333',      u'wac_minc_gt_3333',
              u'rac_total_jobs',        u'rac_jobs_lt_29',
              u'rac_jobs_30_54',        u'rac_jobs_gt_55',
            u'rac_minc_lt_1250',    u'rac_minc_1251_3333',
            u'rac_minc_gt_3333'
]]

#Dropping poverty because nans while computing covariance matrix
jam_route_d_std = jam_route_d

# Standardize Value Range

for c in [
        u'average_commute_time', u'average_walk_distance',
        u'number_of_commuters',u'pop',
                       u'_18_to_24',                    u'above_65',
                         u'age',             u'nonwhite',
                  u'nonenglish',    u'income_below_10000',
          u'income_below_15000',                u'income',
                 u'noncitizen',
                 u'foreignborn',        u'wac_total_jobs',
              u'wac_jobs_lt_29',        u'wac_jobs_30_54',
              u'wac_jobs_gt_55',      u'wac_minc_lt_1250',
          u'wac_minc_1251_3333',      u'wac_minc_gt_3333',
              u'rac_total_jobs',        u'rac_jobs_lt_29',
              u'rac_jobs_30_54',        u'rac_jobs_gt_55',
            u'rac_minc_lt_1250',    u'rac_minc_1251_3333',
            u'rac_minc_gt_3333'
]:
    jam_route_d_std[c] = ((jam_route_d_std[c] - jam_route_d_std[c].mean())/jam_route_d_std[c].std())
    



### PCA 

In [39]:
def select_features(data, data_std):
    mean_vec = np.mean(data_std, axis=0)
    cov_mat = (data_std - mean_vec).T.dot((data_std - mean_vec)) / (data_std.shape[0]-1)
    #print('Covariance matrix \n%s' %cov_mat)

    eig_vals, eig_vecs = np.linalg.eig(cov_mat)
    cor_mat1 = np.corrcoef(data_std.T)

    eig_vals, eig_vecs = np.linalg.eig(cor_mat1)

    #print('Eigenvectors \n%s' %eig_vecs)
    #print('\nEigenvalues \n%s' %eig_vals)

    cor_mat2 = np.corrcoef(data.T)

    eig_vals, eig_vecs = np.linalg.eig(cor_mat2)
    u,s,v = np.linalg.svd(data_std.T)

    # Make a list of (eigenvalue, eigenvector) tuples
    eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]

    #eig_pairs.sort()
    #eig_pairs.reverse()
    
    #print('Eigenvalues in descending order:')
    for index, i in enumerate(eig_pairs):
        #if index<=2:
            print "Eigen Value : " + str(i[0])# max(i[1]), np.mean(i[1]), np.std(i[1]), min(i[1]) )
            print data_std.columns[np.abs(i[1]) > 0.25]
    

### Feature Selection

#### Chinatown

In [16]:
select_features(china_data_o, china_data_o_std)

Eigen Value : 9.47147971294
Index([u'average_commute_time',  u'number_of_commuters',
             u'rac_total_jobs',       u'rac_jobs_lt_29',
             u'rac_jobs_30_54',       u'rac_jobs_gt_55',
           u'rac_minc_gt_3333'],
      dtype='object')
Eigen Value : 6.86559700063
Index([u'wac_total_jobs', u'wac_jobs_lt_29', u'wac_jobs_30_54',
       u'wac_jobs_gt_55', u'wac_minc_lt_1250', u'wac_minc_1251_3333',
       u'wac_minc_gt_3333'],
      dtype='object')
Eigen Value : 3.46025441228
Index([u'_18_to_24', u'nonwhite', u'income_below_10000', u'income_below_15000',
       u'income', u'rac_minc_1251_3333'],
      dtype='object')


In [17]:
select_features(china_data_d, china_data_d_std)

Eigen Value : 9.47147971294
Index([u'average_commute_time',  u'number_of_commuters',
             u'rac_total_jobs',       u'rac_jobs_lt_29',
             u'rac_jobs_30_54',       u'rac_jobs_gt_55',
           u'rac_minc_gt_3333'],
      dtype='object')
Eigen Value : 6.86559700063
Index([u'wac_total_jobs', u'wac_jobs_lt_29', u'wac_jobs_30_54',
       u'wac_jobs_gt_55', u'wac_minc_lt_1250', u'wac_minc_1251_3333',
       u'wac_minc_gt_3333'],
      dtype='object')
Eigen Value : 3.46025441228
Index([u'_18_to_24', u'nonwhite', u'income_below_10000', u'income_below_15000',
       u'income', u'rac_minc_1251_3333'],
      dtype='object')


#### Flatbush

In [18]:
select_features(flat_route_o, flat_route_o_std)

Eigen Value : 9.27159583158
Index([u'number_of_commuters',      u'rac_total_jobs',      u'rac_jobs_lt_29',
            u'rac_jobs_30_54',      u'rac_jobs_gt_55',    u'rac_minc_gt_3333'],
      dtype='object')
Eigen Value : 5.20147115068
Index([u'income_below_10000', u'income_below_15000', u'income',
       u'rac_minc_lt_1250', u'rac_minc_1251_3333'],
      dtype='object')
Eigen Value : 4.57273671815
Index([u'age', u'income_below_15000', u'wac_total_jobs', u'wac_jobs_gt_55',
       u'wac_minc_lt_1250', u'wac_minc_1251_3333'],
      dtype='object')


In [36]:
select_features(flat_route_d, flat_route_d_std)

Eigen Value : 10.7437679117
Index([u'number_of_commuters',      u'rac_total_jobs',      u'rac_jobs_lt_29',
            u'rac_jobs_30_54',      u'rac_jobs_gt_55',    u'rac_minc_gt_3333'],
      dtype='object')
Eigen Value : 6.32550227286
Index([u'wac_total_jobs', u'wac_jobs_lt_29', u'wac_jobs_30_54',
       u'wac_jobs_gt_55', u'wac_minc_1251_3333', u'wac_minc_gt_3333',
       u'rac_minc_1251_3333'],
      dtype='object')
Eigen Value : 3.27711972963
Index([u'_18_to_24', u'nonenglish', u'income_below_10000',
       u'income_below_15000', u'income', u'rac_minc_1251_3333'],
      dtype='object')


#### Jamaica

In [37]:
select_features(jam_route_o, jam_route_o_std)

Eigen Value : 8.7083244658
Index([u'pop', u'rac_total_jobs', u'rac_jobs_lt_29', u'rac_jobs_30_54',
       u'rac_jobs_gt_55', u'rac_minc_lt_1250', u'rac_minc_1251_3333'],
      dtype='object')
Eigen Value : 6.63481701088
Index([u'wac_total_jobs', u'wac_jobs_lt_29', u'wac_jobs_30_54',
       u'wac_jobs_gt_55', u'wac_minc_lt_1250', u'wac_minc_1251_3333',
       u'wac_minc_gt_3333'],
      dtype='object')
Eigen Value : 3.74096997345
Index([u'_18_to_24', u'age', u'nonwhite', u'income_below_10000',
       u'income_below_15000', u'income'],
      dtype='object')


In [40]:
select_features(jam_route_d, jam_route_d_std)

Eigen Value : 9.30963978239
Index([u'_18_to_24', u'income', u'rac_minc_1251_3333'], dtype='object')
Eigen Value : 8.32262706125
Index([u'average_commute_time',   u'wac_minc_1251_3333',
             u'rac_total_jobs',       u'rac_jobs_lt_29',
             u'rac_jobs_30_54',       u'rac_jobs_gt_55',
           u'rac_minc_gt_3333'],
      dtype='object')
Eigen Value : 2.82734073126
Index([u'above_65', u'age', u'nonwhite', u'nonenglish', u'noncitizen'], dtype='object')
Eigen Value : 2.12859892641
Index([u'nonenglish', u'income_below_10000', u'income_below_15000',
       u'foreignborn', u'wac_minc_lt_1250', u'rac_minc_gt_3333'],
      dtype='object')
Eigen Value : 1.71153520575
Index([u'above_65', u'income_below_10000', u'noncitizen', u'foreignborn'], dtype='object')
Eigen Value : 0.986065557316
Index([ u'average_commute_time', u'average_walk_distance',
                  u'nonenglish',           u'foreignborn',
            u'wac_minc_gt_3333'],
      dtype='object')
Eigen Value : 0.80411136

#### Jamaica Destinations f
Eigen Value : 9.30963978239

Index([u'_18_to_24', u'income', u'rac_minc_1251_3333'], dtype='object')


Eigen Value : 8.32262706125

Index([u'average_commute_time',   u'wac_minc_1251_3333',
             u'rac_total_jobs',       u'rac_jobs_lt_29',
             u'rac_jobs_30_54',       u'rac_jobs_gt_55',
           u'rac_minc_gt_3333'],
      dtype='object')


Eigen Value : 2.82734073126

Index([u'above_65', u'age', u'nonwhite', u'nonenglish', u'noncitizen'], dtype='object')

In [24]:
len(flat_route_d)

74

In [23]:
jam_route_d

,average_commute_time,average_walk_distance,number_of_commuters,pop,_18_to_24,above_65,age,nonwhite,nonenglish,income_below_10000,...,wac_minc_lt_1250,wac_minc_1251_3333,wac_minc_gt_3333,rac_total_jobs,rac_jobs_lt_29,rac_jobs_30_54,rac_jobs_gt_55,rac_minc_lt_1250,rac_minc_1251_3333,rac_minc_gt_3333
549,-1.008879,-0.981159,1.131581,1.613843,1.303538,-1.113258,-0.465591,0.877299,1.476367,1.910251,...,-0.045036,-0.306508,-0.823455,1.201772,1.808583,0.871037,1.012056,2.033115,2.132407,-0.495436
600,0.452073,-0.161629,0.415848,1.294142,0.301595,0.623082,0.149936,1.040402,-1.371391,0.649961,...,-0.432640,-0.537067,-0.734164,0.434989,0.064670,0.407753,0.890122,-0.048294,0.995152,0.013947
601,0.644342,-1.360873,0.654426,0.699547,0.939195,1.930695,1.452012,1.007040,-1.161997,0.816414,...,-0.144706,-0.320070,0.114093,0.546991,0.206363,0.318839,1.576003,0.421043,0.436273,0.373244
733,-1.892309,-0.893040,0.845288,0.025027,-1.337948,-1.199003,0.078914,-0.935373,-1.069863,-0.657887,...,1.250670,2.731444,1.379038,2.106404,1.634192,2.485513,0.798671,1.196470,-0.220086,2.956544
916,-1.511240,-0.771878,2.168310,1.135503,0.787385,0.172919,0.623418,1.203505,-1.329512,-0.182306,...,-0.299747,-0.347195,-0.823455,0.449348,0.358955,0.548142,0.097548,0.716930,0.546749,0.009399
1339,0.282141,0.835626,-0.230480,-0.597420,1.030281,0.194356,-0.299872,0.402816,0.655543,0.103043,...,-0.355119,-0.808312,-0.927627,-0.305947,0.151866,-0.280154,-0.923654,-0.231948,0.267309,-0.568205
1342,-0.174369,-0.099900,-0.217467,0.092842,-0.002024,-0.705969,-1.104792,-0.712960,0.747676,-0.491434,...,-0.598756,-0.930373,-0.942508,-0.466771,-0.338610,-0.429903,-0.603576,-0.252354,0.124340,-0.713743
1343,-0.467485,-0.544289,0.233662,-0.155410,-0.305643,-0.898895,-1.246836,-0.071667,1.007324,0.982868,...,0.021411,0.520792,-0.243068,0.354577,0.980225,0.220567,-0.207289,0.747539,0.761203,-0.304417
1344,1.548279,1.065740,-1.197805,-1.630393,0.939195,-0.963204,-0.205176,0.002471,0.094367,-1.561491,...,-0.521235,-0.537067,-0.957390,-1.342684,-1.068873,-1.258199,-1.533326,-1.037984,-0.830954,-1.082136
1345,-0.129955,-0.426719,0.185946,0.050458,1.030281,-1.241876,-2.004408,-0.023477,0.312137,-0.681666,...,-0.565533,-0.794750,1.155813,-0.038866,0.337157,-0.102328,-0.344465,0.023127,0.605236,-0.495436


# Not Using this :


In [ ]:
# Not Using this

matrix_w = np.hstack((eig_pairs[0][1].reshape(28,1),
                      eig_pairs[1][1].reshape(28,1)))

#print('Matrix W:\n', matrix_w)
Y = data_std.dot(matrix_w)
from sklearn.decomposition import PCA as sklearnPCA
sklearn_pca = sklearnPCA(n_components=2)
Y_sklearn = sklearn_pca.fit_transform(data_std)
from sklearn.decomposition import SparsePCA as sparsePCA
sparse_pca = sparsePCA(n_components=4)
sparse_pca.fit(data_std)
Y_sparse = sparse_pca.transform(data_std)